In [ ]:
from transformers import pipeline
import torch
import pandas as pd
import re, unicodedata

if torch.cuda.is_available():
    print("GPU disponible !")
    print(f"Nom du GPU : {torch.cuda.get_device_name(0)}")
else:
    print("GPU non disponible, utilisation du CPU.")

c:\Users\Fournier\Documents\Cours CS\fillière recherche\psychological_analysis_of_breast_cancer_patients\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU non disponible, utilisation du CPU.


In [2]:
topics_name = "Chemotherapy - Before, During, and After"
df = pd.read_csv("../data/scraped_data_in_treatment/" + topics_name + ".csv")

In [3]:
dico_thread_conversation = df.groupby('thread_title')['comment_text'].apply(list).to_dict()
print(next(iter(dico_thread_conversation)))
print(next(iter(dico_thread_conversation.values()))[1])

1St Chemo Fulphila Injection
Hi @sukhbrar -Sorry to hear your Mom is going through chemo. Nice that you can be her support person.Unfortunately, diarrhea and/or constipation is very common with chemo. Everyone is different, but when I was on TC, the first week was the worst, the 2nd week I started feeling better and the 3rd week I felt pretty good, just in time to get ready for my next treatment. 🙄If you want to read more about people's experiences with TC, there is a thread devoted to it here:
    
        https://community.breastcancer.org/en/discussion/697783/anyone-on-just-taxotere-and-cytoxan
    
Anyone on just Taxotere and Cytoxan?DianeBJan 2, 2008I just started a regimen of taxotere and cytoxan for four rounds. I really need to talk to someone who has been on just this regimen. My first session did not go well. It dropped my sodium levels too low and I had a couple of seizures. They kept me in the hospital for two nights. I have been achey and weak since.  Does it get better? I

In [4]:
list_drug_breast_cancer  = [
    "Abemaciclib",
    "Abraxane (Paclitaxel Albumin-stabilized Nanoparticle Formulation)",
    "Ado-Trastuzumab Emtansine",
    "Afinitor (Everolimus)",
    "Afinitor Disperz (Everolimus)",
    "Alpelisib",
    "Anastrozole",
    "Aredia (Pamidronate Disodium)",
    "Arimidex (Anastrozole)",
    "Aromasin (Exemestane)",
    "Capecitabine",
    "Capivasertib",
    "Cyclophosphamide",
    "Datroway (Datopotamab deruxtecan-dlnk)",
    "Datopotamab deruxtecan-dlnk",
    "Docetaxel",
    "Doxorubicin Hydrochloride",
    "Elacestrant Dihydrochloride",
    "Ellence (Epirubicin Hydrochloride)",
    "Enhertu (Fam-Trastuzumab Deruxtecan-nxki)",
    "Epirubicin Hydrochloride",
    "Eribulin Mesylate",
    "Everolimus",
    "Exemestane",
    "5-FU (Fluorouracil Injection)",
    "Fam-Trastuzumab Deruxtecan-nxki",
    "Fareston (Toremifene)",
    "Faslodex (Fulvestrant)",
    "Femara (Letrozole)",
    "Fluorouracil Injection",
    "Fulvestrant",
    "Gemcitabine Hydrochloride",
    "Gemzar (Gemcitabine Hydrochloride)",
    "Goserelin Acetate",
    "Halaven (Eribulin Mesylate)",
    "Herceptin Hylecta (Trastuzumab and Hyaluronidase-oysk)",
    "Herceptin (Trastuzumab)",
    "Ibrance (Palbociclib)",
    "Inavolisib",
    "Infugem (Gemcitabine Hydrochloride)",
    "Itovebi (Inavolisib)",
    "Ixabepilone",
    "Ixempra (Ixabepilone)",
    "Kadcyla (Ado-Trastuzumab Emtansine)",
    "Keytruda (Pembrolizumab)",
    "Kisqali (Ribociclib Succinate)",
    "Kisqali Femara Co-Pack (Ribociclib Succinate and Letrozole)",
    "Lapatinib Ditosylate",
    "Letrozole",
    "Lynparza (Olaparib)",
    "Margenza (Margetuximab-cmkb)",
    "Margetuximab-cmkb",
    "Megestrol Acetate",
    "Methotrexate Sodium",
    "Neratinib Maleate",
    "Nerlynx (Neratinib Maleate)",
    "Olaparib",
    "Orserdu (Elacestrant Dihydrochloride)",
    "Paclitaxel",
    "Paclitaxel Albumin-stabilized Nanoparticle Formulation",
    "Palbociclib",
    "Pamidronate Disodium",
    "Pembrolizumab",
    "Perjeta (Pertuzumab)",
    "Pertuzumab",
    "Pertuzumab, Trastuzumab, and Hyaluronidase-zzxf",
    "Phesgo (Pertuzumab, Trastuzumab, and Hyaluronidase-zzxf)",
    "Piqray (Alpelisib)",
    "Ribociclib Succinate",
    "Ribociclib Succinate and Letrozole",
    "Sacituzumab Govitecan-hziy",
    "Soltamox (Tamoxifen Citrate)",
    "Talazoparib Tosylate",
    "Talzenna (Talazoparib Tosylate)",
    "Tamoxifen Citrate",
    "Taxotere (Docetaxel)",
    "Tecentriq (Atezolizumab)",
    "Tepadina (Thiotepa)",
    "Thiotepa",
    "Toremifene",
    "Trastuzumab",
    "Trastuzumab and Hyaluronidase-oysk",
    "Trodelvy (Sacituzumab Govitecan-hziy)",
    "Truqap (Capivasertib)",
    "Tucatinib",
    "Tukysa (Tucatinib)",
    "Tykerb (Lapatinib Ditosylate)",
    "Verzenio (Abemaciclib)",
    "Vinblastine Sulfate",
    "Xeloda (Capecitabine)",
    "Zoladex (Goserelin Acetate)"
]

In [ ]:
# def find_drug_mentions(list_drug, dico_thread_conv):
    drug_mentions = {drug: [] for drug in list_drug}

    for drug in list_drug:
        for thread_name, conversations in dico_thread_conv.items():
            if drug.lower() in thread_name.lower():
                drug_mentions[drug].append(message for message in conversations)
                continue
            for message in conversations:
                if isinstance(message,float):
                    continue
                if drug.lower() in message.lower():
                    drug_mentions[drug].append(message)

    return drug_mentions


In [28]:
def normalize_text(s):
    """Supprime accents, met en minuscules et remplace les caractères non alphanumériques par des espaces."""
    if s is None:
        return ''
    s = str(s)
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')  # retire les diacritiques
    s = s.lower()
    s = re.sub(r'[^a-z0-9]+', ' ', s)  # garde lettres et chiffres ASCII
    return s.strip()

def _build_drug_regex(drug):
    norm = normalize_text(drug)
    if not norm:
        return None
    tokens = norm.split()
    # autorise plusieurs espaces entre tokens (ex: "dolo 650" correspondra à "dolo-650")
    pattern = r'\b' + r'\s+'.join(map(re.escape, tokens)) + r'\b'
    return re.compile(pattern, re.IGNORECASE)

def find_drug_mentions(list_drug, dico_thread_conv):
    """Retourne {drug: [messages]} — messages uniques, dans l'ordre rencontré."""
    drug_mentions = {drug: [] for drug in list_drug}
    for drug in list_drug:
        regex = _build_drug_regex(drug)
        if regex is None:
            continue
        seen = set()
        for thread_name, conversations in dico_thread_conv.items():
            # accepte conversations comme str ou liste
            if isinstance(conversations, str):
                conversations = [conversations]
            # si le nom du thread contient le médicament, on ajoute tous les messages du thread
            if regex.search(normalize_text(thread_name)):
                for msg in conversations:
                    m = str(msg).strip()
                    if m and m not in seen:
                        drug_mentions[drug].append(msg)
                        seen.add(m)
                continue
            # sinon on cherche message par message
            for msg in conversations:
                m = str(msg).strip()
                if not m or m in seen:
                    continue
                if regex.search(normalize_text(m)):
                    drug_mentions[drug].append(msg)
                    seen.add(m)
    return drug_mentions

In [29]:
dico_drug_mention = find_drug_mentions(list_drug_breast_cancer,dico_thread_conversation)

In [32]:
print(dico_drug_mention)

drugs_with_mentions = [drug for drug, messages in dico_drug_mention.items() if messages]

print("Médicaments trouvés :", drugs_with_mentions)

{'Abemaciclib': [], 'Abraxane (Paclitaxel Albumin-stabilized Nanoparticle Formulation)': [], 'Ado-Trastuzumab Emtansine': [], 'Afinitor (Everolimus)': [], 'Afinitor Disperz (Everolimus)': [], 'Alpelisib': [], 'Anastrozole': ['Good morning, chemo sisters.  It\'s wonderful to hear from you.  Tomorrow will be the 1 year anniversary of my lumpectomy.  I remember the uncertainty was crushing and I was just on the edge of tears most of the day before.  But I had the best BS, very kind and reassuring.  After the pathology came back and we learned I needed the full-Monty (chemo, radiation,  1 year of Herceptin and 5-10 years of AI), I remember coming here and finding this great group.  It was a long, hard winter of chemo, every Friday for 18 weeks with every 3rd Friday a full cocktail.  I was surprised by my reaction the other day as the weather got cold and I dropped my infinity scarf around my shoulders.  I wore it constantly last winter and my whole body shook as I was reminded of how rotte